## Merge results from T2, ADC, DWI 2000

In [ ]:
import os
import time

from tqdm import tqdm
import numpy as np
import SimpleITK as sitk

In [ ]:
#input_root =[r'D:\Annotator\B78_1st_round\MaskEdited', r'D:\Annotator\B78_2st_round\MaskEdited']
#output_root = [r'B78_reader1_output, 'r'B78_reader2_output']
#reader_name = ['KQQ','LF','WHC', 'YJY', 'ZHY','WL','WZ']
reader_name = ['WJ1', 'WJ2', 'WTL1', 'WTL2', 'XPY1','XPY2']
aided = False
case_list = ['P' + (3-len(str(x)))*'0' + str(x) for x in range(1,181)]

In [ ]:
error_list = []
for roundx in ['1st']:
    input_root = rf'D:\Annotator\B78_GT\MaskEdited'
    output_root = rf'D:\dataset\B78_GT_output'
    for reader in reader_name:
        print(f'Now process {reader}\'s data')
        #time.sleep(1)
        reader_annotations = os.path.join(input_root, reader)
        for case in tqdm(case_list):
            image_status = False
            mask_count = 0
            for folder in os.listdir(reader_annotations):
                if folder.startswith(case):
                    image = sitk.ReadImage(os.path.join(reader_annotations, folder, folder+'.mhd'))
                    image_array = sitk.GetArrayFromImage(image)
                    if np.max(image_array) > 5:
                        error_list.append([os.path.join(reader_annotations, folder), np.max(image_array)])
                    if np.max(image_array) > 0:
                        mask_count += 1
                    if not image_status:
                        temp_mask = image_array
                        image_origin = image.GetOrigin()
                        image_spacing = image.GetSpacing()
                        image_direction = image.GetDirection()
                        image_status = True
                    else:
                        temp_mask += image_array
            if mask_count > 1:
                error_list.append(os.path.join(reader_annotations, case))
            mask_image = sitk.GetImageFromArray(temp_mask)
            mask_image.SetOrigin(image_origin)
            mask_image.SetSpacing(image_spacing)
            mask_image.SetDirection(image_direction)
            output_folder = os.path.join(output_root, reader)
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            sitk.WriteImage(mask_image, os.path.join(output_folder, case+'.mhd'))              

In [ ]:
for error in error_list:
    print(error)

## Extract Case Info

In [ ]:
import os

import numpy as np
import SimpleITK as sitk
import pandas as pd

In [ ]:
input_root = r''
reader_list = ['WJ1', 'WJ2', 'WTL1', 'WTL2', 'XPY1','XPY2']

In [ ]:
for i in range(2,3):
    input_root = rf'D:\dataset\B78_GT_output'
    readers_results = rf'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\Projects\Prostate MR\innovation_path\pilot_study\readers_annotation\gt_annotation.xlsx'
    readers_df = pd.read_excel(readers_results)
    for reader in reader_list:
        reader_folder = os.path.join(input_root, reader)
        
        for idx, row in readers_df.iterrows():
            case = row['PID']
            image = sitk.ReadImage(os.path.join(input_root, reader, case+'.mhd'))
            image_array = sitk.GetArrayFromImage(image)
            readers_df[reader].iloc[idx] = np.max(image_array)
    readers_df.to_excel(readers_results, index=False)

## Extract Lesion Info

In [1]:
import os
import SimpleITK as sitk
import pandas as pd
import numpy as np
from tqdm import tqdm
from skimage.measure import label, regionprops

In [2]:
reader_name = ['KQQ','LF','WHC', 'YJY', 'ZHY','WL','WZ']
case_list = ['P' + (3-len(str(x)))*'0' + str(x) for x in range(1,181)]


In [29]:
def get_lesion_info(mask, area_thres=50):
    lesion_list = []
    image = sitk.ReadImage(mask)
    mask_array = sitk.GetArrayFromImage(image)
    #print(mask_array.shape)
    labels = label(mask_array, connectivity=3)
    regions = regionprops(labels)
    for region in regions:
        if region.area < area_thres:
            #print(region.area)
            continue
        if region.area < 100:
            print(mask, region.area)
        centroid = np.round(region.centroid)
        phys_centroid = np.round(image.TransformContinuousIndexToPhysicalPoint(centroid[::-1]),4)
        #print(centroid, phys_centroid)
        score = mask_array[region.coords[0][0], region.coords[0][1], region.coords[0][2]]
        lesion_list.append([phys_centroid, score])
    return lesion_list

In [30]:
lesion_info = dict()
cols = ['Round','Reader','PID', 'Lesion_Idx', 'Phys_Z', 'Phys_Y', 'Phys_X', 'Score']
for col in cols:
    lesion_info.setdefault(col, [])
for roundx in [1,2]:
    input_root = rf'D:\dataset\B78_reader{roundx}_output'
    for Reader in reader_name:
        reader_folder = os.path.join(input_root, Reader)
        for case in tqdm(case_list):
            case_mask = os.path.join(reader_folder, case+'.mhd')
            case_lesion = get_lesion_info(case_mask)
            lesion_index = 0
            
            if case_lesion:
                for lesion in case_lesion:
                    lesion_index += 1
                    lesion_info['Round'].append(roundx)
                    lesion_info['Reader'].append(Reader)
                    lesion_info['PID'].append(case)
                    lesion_info['Lesion_Idx'].append(lesion_index)
                    lesion_info['Phys_Z'].append(lesion[0][0])
                    lesion_info['Phys_Y'].append(lesion[0][1])
                    lesion_info['Phys_X'].append(lesion[0][2])
                    lesion_info['Score'].append(lesion[1])
            else:
                lesion_info['Round'].append(roundx)
                lesion_info['Reader'].append(Reader)
                lesion_info['PID'].append(case)
                for col in ['Lesion_Idx','Phys_Z', 'Phys_Y', 'Phys_X', 'Score']:
                    lesion_info[col].append('')
    
                       

  8%|██████▎                                                                          | 14/180 [00:00<00:08, 19.48it/s]

D:\dataset\B78_reader1_output\KQQ\P012.mhd 52


 65%|████████████████████████████████████████████████████                            | 117/180 [00:05<00:01, 33.74it/s]

D:\dataset\B78_reader1_output\KQQ\P110.mhd 56


  1%|▉                                                                                 | 2/180 [00:00<00:10, 17.28it/s]

...

 59%|███████████████████████████████████████████████▌                                | 107/180 [00:04<00:02, 31.23it/s]

D:\dataset\B78_reader2_output\WZ\P102.mhd 86
D:\dataset\B78_reader2_output\WZ\P103.mhd 77


 64%|███████████████████████████████████████████████████                             | 115/180 [00:05<00:01, 32.93it/s]

D:\dataset\B78_reader2_output\WZ\P111.mhd 84


100%|████████████████████████████████████████████████████████████████████████████████| 180/180 [00:06<00:00, 25.73it/s]


In [25]:
lesion_df = pd.DataFrame(lesion_info)
lesion_df.to_excel('pirads_reader.xlsx', index=False)

In [9]:
test_file = r'D:\dataset\B78_reader1_output\KQQ\P003.mhd'
test_image = sitk.ReadImage(test_file)
test_array = sitk.GetArrayFromImage(test_image)

In [15]:
labels = label(test_array, connectivity=3)
regions = regionprops(labels)

In [17]:
for region in regions:
    if region.area < 30:
        continue
    print(region.area)

13331
